# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,2.79,81,10,10.61,TF,1722713302
1,1,luderitz,-26.6481,15.1594,12.68,94,14,4.86,NaN,1722713303
2,2,adamstown,-25.0660,-130.1015,20.83,72,19,6.30,PN,1722713304
3,3,bethel,41.3712,-73.4140,32.79,65,75,2.06,US,1722713087
4,4,bikenibeu village,1.3673,173.1241,27.98,83,20,2.57,KI,1722713306


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Humidity", "City"]
)
# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_data_df = city_data_df[
    (city_data_df['Max Temp'] <= 27) &
    (city_data_df['Max Temp'] >= 19) &
    (city_data_df['Wind Speed'] < 10) &
    (city_data_df['Cloudiness'] < 15) &
    (city_data_df['Humidity'] <= 75)
    ]

# Drop any rows with null values
narrow_data_df.dropna(axis=0, inplace=True)

# Display sample data
narrow_data_df

C:\Users\nbart.DESKTOP-3OF7M8N\AppData\Local\Temp\ipykernel_19292\902436011.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  narrow_data_df.dropna(axis=0, inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,kittila,67.6647,24.8936,19.08,55,0,1.54,FI,1722713315
30,30,palestine,31.9216,35.2033,23.30,66,0,1.25,PS,1722713338
54,54,smithers,54.7804,-127.1743,23.18,33,0,2.06,CA,1722713235
61,61,chissamba,-12.1667,17.3333,20.04,14,0,1.29,AO,1722713373
68,68,zaio,34.9428,-2.7329,26.90,41,0,2.84,MA,1722713299
69,69,shizuishan,39.2333,106.7694,25.21,43,0,4.41,CN,1722713382
75,75,karratha,-20.7377,116.8463,20.27,75,0,3.02,AU,1722713389
151,151,palmeira,-25.4294,-50.0064,22.25,53,0,2.53,BR,1722713482
197,197,urzhar,47.0930,81.6294,22.64,47,1,3.69,KZ,1722713539
204,204,colonia,50.9333,6.9500,22.75,75,0,4.12,DE,1722713396


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,kittila,FI,67.6647,24.8936,55,
30,palestine,PS,31.9216,35.2033,66,
54,smithers,CA,54.7804,-127.1743,33,
61,chissamba,AO,-12.1667,17.3333,14,
68,zaio,MA,34.9428,-2.7329,41,
69,shizuishan,CN,39.2333,106.7694,43,
75,karratha,AU,-20.7377,116.8463,75,
151,palmeira,BR,-25.4294,-50.0064,53,
197,urzhar,KZ,47.0930,81.6294,47,
204,colonia,DE,50.9333,6.9500,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key,
    'categories': 'accommodation.hotel',
    'limit': 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    long = row['Lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params['filter'] = f'circle:{long},{lat},{radius}'
    params['bias'] = f'proximity:{long},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kittila - nearest hotel: Hotelli Kittilä
palestine - nearest hotel: Retno Hotel
smithers - nearest hotel: Sunshine Inn Hotel
chissamba - nearest hotel: No hotel found
zaio - nearest hotel: Bilal بلال
shizuishan - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
palmeira - nearest hotel: Hotel Vereda
urzhar - nearest hotel: Гостиница Шанырак
colonia - nearest hotel: Wasserturm Hotel Cologne
itaparica - nearest hotel: Pousada El Shaday
huanta - nearest hotel: Hotel Imperial
bodo - nearest hotel: Zefyr hotell
kolonia - nearest hotel: Wasserturm Hotel Cologne
shitanjing - nearest hotel: No hotel found
lerma - nearest hotel: DC
touros - nearest hotel: Pousada Atlântico
thulamahashi - nearest hotel: No hotel found
nouadhibou - nearest hotel: Noadhibou Guesthouse
santo antonio - nearest hotel: Pousada Onça Puma
sinazongwe - nearest hotel: Lake View Lodge
damavand - nearest hotel: Avishan Hotel
chardara - nearest hotel: Бақдаулет


,City,Country,Lat,Lng,Humidity,Hotel Name
11,kittila,FI,67.6647,24.8936,55,Hotelli Kittilä
30,palestine,PS,31.9216,35.2033,66,Retno Hotel
54,smithers,CA,54.7804,-127.1743,33,Sunshine Inn Hotel
61,chissamba,AO,-12.1667,17.3333,14,No hotel found
68,zaio,MA,34.9428,-2.7329,41,Bilal بلال
69,shizuishan,CN,39.2333,106.7694,43,No hotel found
75,karratha,AU,-20.7377,116.8463,75,Karratha International Hotel
151,palmeira,BR,-25.4294,-50.0064,53,Hotel Vereda
197,urzhar,KZ,47.0930,81.6294,47,Гостиница Шанырак
204,colonia,DE,50.9333,6.9500,75,Wasserturm Hotel Cologne


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    hover_cols= ['Hotel Name', 'Country'],
    color= 'City',
    size= 'Humidity',
    alpha= .7
     
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)